In [1]:
#load packages
import papermill as pm                               #to parameterize and execute across jupyter notebooks
import pandas as pd
import os
from concurrent.futures import ThreadPoolExecutor    #for parallelization
import numpy as np
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from tabulate import tabulate
import plotly.graph_objects as go
from datetime import datetime
import os
import signal
import sys
from datetime import datetime

#custom functions
from functions import (
    solve_single_UC,
    save_checkpoint,
    init_state
)

In [2]:
file_path = ''
subfolder_name = ''

In [3]:
# Parameters
file_path = "/Users/shinychoudhury/Library/CloudStorage/Dropbox/TYNAN_Lab/Research/5_NuclearReactor_technical/physics-informed-nuclear-fleet-unit-commitment/Config/mode3_pmin0.2_PHSduration4_costfrac1.0_kinit1.205_startday0_endday360_refuelspan10.txt"
subfolder_name = "/Users/shinychoudhury/Library/CloudStorage/Dropbox/TYNAN_Lab/Research/5_NuclearReactor_technical/physics-informed-nuclear-fleet-unit-commitment/results/mode3_pmin0.2_PHSduration4_costfrac1.0_kinit1.205_startday0_endday360_refuelspan10"


In [4]:
# Path to the text file
# file_path = 'Config.txt'

# Open the file and read its contents
with open(file_path, 'r') as file:
    for line in file:
        # Strip whitespace and ignore comments or empty lines
        line = line.strip()
        if line and not line.startswith('#'):
            # Directly execute the line to set variables
            exec(line)
            print(line)

Wind_Cap = 8000
Solar_Cap = 12000
Nuclear_Cap = 5000
PHS_Cap = 6000
horizon = 72
load_percentage = 1.0
PHS_percentage = 0.1
model_name = "ap1000"
VRE_percentage = 0.2
mAP300 = 0.001137
mAP100 = 0.001137
q = 0.11
nuclear_unit = 0
pmin = 0.2
PHS_duration = 4
cost_frac = 1.0
k_init_ap1000 = 1.205
mAP1000 = 0.001137
start_day = 0
end_day = 360
refuel_span = 10


In [5]:
#path+load data+build basic gen_df
path=os.getcwd()

gen_info = pd.read_csv(path+"/data/"+"Generators_data_nuclear.csv")   #has data on cost and engineering charactristics of all the different kinds of generators 
fuels = pd.read_csv(path+"/data/"+"Fuels_data.csv") #has details on fuel costs
gen_variable = pd.read_csv(path+"/data/"+"Generators_variability_nuclear.csv")  #CF for all generators, added geothermal
solar_wind_variablity_yearly=pd.read_csv(path+"/data/"+"wind_solar_variability_ERCOT.csv")
load_yearly=pd.read_csv(path+"/data/"+"ERCOT_south_load_wind_2021_2023.csv")
load_yearly.Demand=load_yearly.Demand


#build a temp gen_df from original generator info table for use to built init kinf table
n=0                                            #random choice
T_period = range(n * 24 + 1, (n + 1) * 24 + 1)    #isolate hours and other relevant data for day 'n'
loads = load_yearly[load_yearly['Hour'].isin(T_period)].reset_index(drop=True)
loads['Hour'] = range(1, 25)  

# (1) Rename all columns to lowercase (by convention)
for data in [gen_info, fuels, loads, gen_variable]:
    data.columns = data.columns.str.lower()
    
# (2) Keep columns relevant to our UC model 
gen_info = gen_info.iloc[:, 0:26]

#(3) Load in fuel costs and add to data frame
gen_df = gen_info.merge(fuels, on="fuel", how="outer").sort_values(by="r_id").reset_index(drop=True)
gen_df.rename({'cost_per_mmbtu': 'fuel_cost'}, axis=1, inplace=True)

#(4) Create "is_variable" column to indicate if this is a variable generation source (e.g. wind, solar)
gen_df["is_variable"] = False
gen_df.loc[
    gen_df["resource"].isin(["onshore_wind_turbine","small_hydroelectric","solar_photovoltaic"]), 
    "is_variable"] = True

#(5) Create full name of generator (including geographic location and cluster number)
#  for use with variable generation dataframe
gen_df["gen_full"] = gen_df["region"] + "_" + gen_df["resource"] + "_" + gen_df["cluster"].astype(str)
gen_df["gen_full"] = gen_df["gen_full"].str.lower()

# (6) Remove generators with no capacity (e.g. new build options that we'd use if this was capacity expansion problem)
gen_df = gen_df[gen_df["existing_cap_mw"] > 0].reset_index(drop=True)

#Isolate wind and solar hourly CF factors for 'n' or 'T_period'
wind_solar_T_period=solar_wind_variablity_yearly[solar_wind_variablity_yearly['Hour'].isin(T_period)].reset_index(drop=True)
hour_index=wind_solar_T_period["Hour"]  #to be used later in building transposed_df

#replace wind and solar hourly CF factors in gen_variable
gen_variable["WEC_SDGE_onshore_wind_turbine_1.0"]=wind_solar_T_period["Wind"]
gen_variable["WEC_SDGE_solar_photovoltaic_1.0"]=wind_solar_T_period["Solar"]

#take all capacities from CEM
gen_df.loc[gen_df['resource'] == 'onshore_wind_turbine', 'existing_cap_mw'] = int(Wind_Cap)
gen_df.loc[gen_df['resource'] == 'solar_photovoltaic',   'existing_cap_mw'] = int(Solar_Cap)
gen_df.loc[gen_df['resource'] == 'ap1000',   'existing_cap_mw'] = int(Nuclear_Cap)
gen_df.loc[gen_df['resource'] == 'hydroelectric_pumped_storage',   'existing_cap_mw'] = int(PHS_Cap)

# Step 1: Filter relevant resources, but exclude all existing 'ap300'
keep_resources = ['hydroelectric_pumped_storage', 'onshore_wind_turbine', 'solar_photovoltaic']
gen_df_filtered = gen_df[gen_df['resource'].isin(keep_resources)].copy()

# Step 2: Get one representative 'ap300' row (you choose how – first, template, etc.)
template_ap1000_row = gen_df[gen_df['resource'] == 'ap1000'].iloc[0].copy()

# Step 3: Count reactors and reassign
cap = gen_df.loc[gen_df['resource'] == 'ap1000', 'existing_cap_mw'].values[0]
repeat_count = int(cap // 1000)

# Step 4: Repeat the ap300 row
ap1000_replicated = pd.DataFrame([template_ap1000_row] * repeat_count)

# Step 5: Combine
gen_df = pd.concat([gen_df_filtered, ap1000_replicated], ignore_index=True)
gen_df = gen_df.assign(existing_cap_mw = np.where(gen_df["resource"]=="ap1000", gen_df["existing_cap_mw"]/repeat_count, gen_df["existing_cap_mw"]))

#rename 'r_id' column depending on how many generators are left
for ii in range(gen_df.shape[0]):
    gen_df['r_id'][ii]=ii+1
    
gen_df['r_id'] = gen_df['r_id'].astype(int)     #this gets built in UC.ipynb independently
gen_df

<ipython-input-5-d919efbcb4f3>:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gen_df['r_id'][ii]=ii+1
<ipython-input-5-d919efbcb4f3>:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gen_df['r_id'][ii]=ii+1
<ipython-input-5-d919efbcb4f3>:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gen_df['r_id'][ii]=ii+1
<ipython-input-5-d919efbcb4f3>:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFram

,r_id,resource,region,existing_cap_mw,var_om_cost_per_mwh,fuel,heat_rate_mmbtu_per_mwh,min_power,ramp_up_percentage,ramp_dn_percentage,start_cost_per_mw,shut_cost_per_mw,up_time,down_time,cluster,fuel_cost,co2_content_tons_per_mmbtu,is_variable,gen_full
0,1,hydroelectric_pumped_storage,WEC_SDGE,6000.0,0.00,NaN,0.00,0.0,0.00,0.00,0.00,0.00,0.0,0.0,1.0,0.0,0.0,False,wec_sdge_hydroelectric_pumped_storage_1.0
1,2,onshore_wind_turbine,WEC_SDGE,8000.0,0.00,NaN,0.00,0.0,1.00,1.00,0.00,0.00,0.0,0.0,1.0,0.0,0.0,True,wec_sdge_onshore_wind_turbine_1.0
2,3,solar_photovoltaic,WEC_SDGE,12000.0,0.00,NaN,0.00,0.0,1.00,1.00,0.00,0.00,0.0,0.0,1.0,0.0,0.0,True,wec_sdge_solar_photovoltaic_1.0
3,4,ap1000,WEC_SDGE,1000.0,4.55,uranium,10.44,0.5,0.25,0.25,107.86,107.86,1.0,1.0,1.0,0.5,0.0,False,wec_sdge_ap1000_1.0
4,5,ap1000,WEC_SDGE,1000.0,4.55,uranium,10.44,0.5,0.25,0.25,107.86,107.86,1.0,1.0,1.0,0.5,0.0,False,wec_sdge_ap1000_1.0
5,6,ap1000,WEC_SDGE,1000.0,4.55,uranium,10.44,0.5,0.25,0.25,107.86,107.86,1.0,1.0,1.0,0.5,0.0,False,wec_sdge_ap1000_1.0
6,7,ap1000,WEC_SDGE,1000.0,4.55,uranium,10.44,0.5,0.25,0.25,107.86,107.86,1.0,1.0,1.0,0.5,0.0,False,wec_sdge_ap1000_1.0
7,8,ap1000,WEC_SDGE,1000.0,4.55,uranium,10.44,0.5,0.25,0.25,107.86,107.86,1.0,1.0,1.0,0.5,0.0,False,wec_sdge_ap1000_1.0


In [6]:
#Initialize all reactors keff values and form a D_point and kinfTable_udt dataframe for tandem run
#Handels "ap1000" or "ap300" based on whichever is being run

MaxReacTableAP1000=pd.read_csv(path+"/data/"+"reacvspminAP1000.csv")
MaxReacTableAP300=pd.read_csv(path+"/data/"+"reacvspminAP300.csv")

deadtime_matrix_ap1000=pd.read_csv(path+"/data/"+"resultmatrix_ap1000.csv")
deadtime_matrix_ap300=pd.read_csv(path+"/data/"+"resultmatrix_ap300.csv")


MaxReacTableAP1000['resource'] = 'ap1000'
MaxReacTableAP300['resource'] = 'ap300'


MaxReacTable = pd.concat([MaxReacTableAP1000, MaxReacTableAP300], ignore_index=True)


kinfTable=pd.DataFrame()
G_thermal = gen_df[gen_df["up_time"] > 0]["r_id"].tolist() # thermal
G = gen_df["r_id"].tolist()

kinfTable['r_id']=G_thermal
# kinfTable['resource']=gen_info[gen_info['r_id'].isin(G_thermal)][['resource']]

kinfTable=pd.DataFrame()
kinfTable = gen_df[gen_df['r_id'].isin(G_thermal)][['r_id', 'resource']].reset_index(drop=True).copy()
kinfTable['keff'] = kinfTable['resource'].apply(lambda x: k_init_ap1000 if x == 'ap1000' else (k_init_ap300 if x == 'ap300' else (k_init_ap100 if x == 'ap100' else None)))
kinfTable['ReacValue']=((kinfTable['keff']-1)/kinfTable['keff'])*(1e5)

# Initialize an empty list to hold the 'p0' values
p0_values = []

# Loop through each row in 'kinfTable'
for index, row in kinfTable.iterrows():
    resource = row['resource']
    reac_value = row['ReacValue']
    # Filter 'MaxReacTable' to only include rows with the same 'resource'
    filtered_max_reac_table = MaxReacTable[MaxReacTable['resource'] == resource]
    
    # Compute the positive differences between 'MaxReactivityXe' and 'ReacValue'
    differences = reac_value - filtered_max_reac_table['MaxReactivityXe']
    positive_differences = differences[differences >= 0]
    
    # Find the row in 'MaxReacTable' with the smallest positive difference
    if not positive_differences.empty:
        min_diff_index = positive_differences.idxmin()
        corresponding_p0 = MaxReacTable.loc[min_diff_index, 'p0']
    else:
        corresponding_p0 = np.nan  # Use NaN if no such value exists
    
    p0_values.append(corresponding_p0)

# Add the 'p0' values as a new column in 'kinfTable'
# p0_values_clean = np.where(np.isnan(p0_values), int(1), p0_values)

# kinfTable['p0'] = p0_values_clean
kinfTable['p0'] = np.where(np.isnan(p0_values), 1, np.maximum(p0_values, pmin))
kinfTable['alpha']=1

kinfTable_udt=kinfTable.copy()

# Initialize a new column 'nearest_k' in the DataFrame kinfTable_udt and fill it with NaN (Not a Number) values.
kinfTable_udt['nearest_k'] = np.nan  
# Initialize D_points DataFrame with 'r_id' from kinfTable_udt
D_points = pd.DataFrame()
D_points['r_id'] = kinfTable_udt['r_id']

# Initialize columns for each of the 8 deadtime values
for i in range(1, len(deadtime_matrix_ap1000)+1):  # Assuming 8 values
    D_points[f'deadtime_value'] = 0

# Iterate through each row of the DataFrame kinfTable_udt.
for idx, row in kinfTable_udt.iterrows():
    # Extract keff and resource ID for the current row.
    keff_value = row['keff']
    r_id_value = row['r_id']
    resource_value = row['resource']  # Assuming the column is named 'resource'

    # Choose the appropriate deadtime_matrix based on the 'resource'.
    if resource_value == 'ap1000':
        deadtime_matrix = deadtime_matrix_ap1000
    elif resource_value == 'ap300':
        deadtime_matrix = deadtime_matrix_ap300
    else:
        print(f"Unknown resource: {resource_value}")
        continue

    # Calculate the absolute differences between keff_value and each k value, rounded to 4 decimal places.
    # k_diffs = {col: round(abs(keff_value - float(col)), 4) for col in deadtime_matrix.columns if col != 'p'}
    k_values_less_than_keff = [float(col) for col in deadtime_matrix.columns if (col != 'p' and float(col) < keff_value)]
        
    # Find the k value that is the largest but still less than keff_value.
    # Handle empty k_values_less_than_keff by finding the last value in deadtime_matrix
    if not k_values_less_than_keff:
        # Extract the last column value which is a float and not 'p'
        last_column = next((col for col in reversed(deadtime_matrix.columns) if col != 'p'), None)
        nearest_k_value = float(last_column) if last_column else float(1)  # Fallback to 1 if no valid last column found
    else:
        # Find the k value that is the largest but still less than keff_value.
        nearest_k_value = max(k_values_less_than_keff)

    # Update 'nearest_k' in kinfTable_udt.
    kinfTable_udt.loc[kinfTable_udt['r_id'] == r_id_value, 'nearest_k'] = nearest_k_value

    # Convert nearest_k_value to string to access the corresponding column in deadtime_matrix.
    column_str = str(int(nearest_k_value)) if nearest_k_value.is_integer() else str(nearest_k_value)
    deadtime_values = deadtime_matrix[column_str].tolist() 

    # for i, val in enumerate(deadtime_values, start=1):
    #     D_points.loc[D_points['r_id'] == r_id_value, f'deadtime_value_{i}'] = int(val)
    D_points.loc[D_points['r_id'] == r_id_value, 'deadtime_value'] = int(deadtime_values[0])

    # for i, val in enumerate(deadtime_values, start=1):
    #     D_points.loc[D_points['r_id'] == r_id_value, f'deadtime_value'] = int(val)


print('kinf_table:\n',kinfTable_udt, '\n \n D_points:\n', D_points)

kinf_table:
    r_id resource   keff     ReacValue   p0  alpha  nearest_k
0     4   ap1000  1.205  17012.448133  0.2      1    1.20462
1     5   ap1000  1.205  17012.448133  0.2      1    1.20462
2     6   ap1000  1.205  17012.448133  0.2      1    1.20462
3     7   ap1000  1.205  17012.448133  0.2      1    1.20462
4     8   ap1000  1.205  17012.448133  0.2      1    1.20462 
 
 D_points:
    r_id  deadtime_value
0     4               6
1     5               6
2     6               6
3     7               6
4     8               6


In [7]:
"""SOLVE AND SAVE intermediate results in the desired format"""

# Global tracker for last completed day
current_day = None

# Directory to store rolling checkpoints
checkpoint_dir = "checkpoint"
os.makedirs(checkpoint_dir, exist_ok=True)

# Initialize all DataFrames 
state = init_state(G, G_thermal, kinfTable_udt, D_points, start_day)

(master_df, GEN_df, SHUT_df, START_df, RESUP_df, RESDN_df, SLOWRES_df,
 master_shut_df, master_commit, master_kinfTable, master_commit_at_len_T,
 run_details, remaining_downtime, commit_at_len_T,
 terminate_loop_indicator, terminate_loop) = (
    state[k] for k in [
        "master_df", "GEN_df", "SHUT_df", "START_df", "RESUP_df", "RESDN_df", "SLOWRES_df",
        "master_shut_df", "master_commit", "master_kinfTable", "master_commit_at_len_T",
        "run_details", "remaining_downtime", "commit_at_len_T",
        "terminate_loop_indicator", "terminate_loop"
    ]
)

for n in range(start_day, end_day + 1):
    remaining_downtime = pd.read_csv(f'remaining_downtime_{n}.csv', index_col=0)
    commit_at_len_T = pd.read_csv(f'commit_at_len_T_{n}.csv', index_col=0)
    kinfTable_udt = pd.read_csv(f'kinfTable_udt_{n}.csv', index_col=0)
    D_points = pd.read_csv(f'D_points_{n}.csv', index_col=0)
    terminate_loop_indicator = pd.read_csv(f'terminate_loop_indicator_{n}.csv', index_col=0)

    print(f'Solved day {n-1}, going to start solving for day {n} if keff of all reactors > 1')

    termination_status = terminate_loop_indicator

    first_run = True if n == start_day else False

    current_day = n

    transposed_df, shut_df, remaining_downtime, commit_result, kinfTable_udt, \
    commit_at_len_T, D_points, GEN_result, SHUT_result, START_result = \
        solve_single_UC(model_name, horizon, load_percentage, PHS_percentage, PHS_duration, n, nuclear_unit,
                        remaining_downtime, kinfTable_udt, commit_at_len_T, D_points, first_run, pmin,
                        VRE_percentage, cost_frac, mAP1000, mAP300, mAP100, Wind_Cap, Solar_Cap, Nuclear_Cap, PHS_Cap, refuel_span)

    run_data = {
        "Day": int(n),
        "Horizon": int(horizon),
        "Load Percentage": float(load_percentage),
        "PHS Percentage": float(PHS_percentage),
        "PHS Duration": float(PHS_duration),
        "Nuclear Unit": str(nuclear_unit),
        "Model Name": str(model_name),
        "Solar Cap": float(Solar_Cap),
        "Wind Cap": float(Wind_Cap),
        "Date of Run": datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    }
    run_details.append(run_data)

    master_df = pd.concat([master_df, transposed_df], ignore_index=True)
    GEN_df = pd.concat([GEN_df, GEN_result], ignore_index=True)
    SHUT_df = pd.concat([SHUT_df, SHUT_result], ignore_index=True)
    START_df = pd.concat([START_df, START_result], ignore_index=True)
    master_shut_df = pd.concat([master_shut_df, shut_df], ignore_index=True)
    master_commit = pd.concat([master_commit, commit_result], ignore_index=True)
    master_kinfTable = pd.concat([master_kinfTable, kinfTable_udt], ignore_index=True)

    # Save checkpoint after each run
    save_checkpoint(n-1, start_day, checkpoint_dir,
                master_df, GEN_df, SHUT_df, START_df,
                master_shut_df, master_commit, master_kinfTable)

# Save final results in formatted output
os.remove(f'kinfTable_udt_{end_day+1}.csv')
os.remove(f'terminate_loop_indicator_{end_day+1}.csv')
os.remove(f'remaining_downtime_{end_day+1}.csv')
os.remove(f'commit_at_len_T_{end_day+1}.csv')
os.remove(f'D_points_{end_day+1}.csv')

today = datetime.now().strftime("%Y-%m-%d_%H-%M")
resource = kinfTable_udt['resource'].iloc[0]
# subfolder_name = f"datetime_{today}_startday_{start_day}_endday_{end_day}_{resource}"
# subfolder_name = subfolder_name
results_path = os.path.join('results', subfolder_name)
os.makedirs(results_path, exist_ok=True)

master_df.to_csv(os.path.join(results_path, 'aggregate_results.csv'))
GEN_df.to_csv(os.path.join(results_path, 'GEN_results.csv'))
SHUT_df.to_csv(os.path.join(results_path, 'SHUT_results.csv'))
START_df.to_csv(os.path.join(results_path, 'START_results.csv'))
master_shut_df.to_csv(os.path.join(results_path, 'shut_count_and_obj_value.csv'))
master_commit.to_csv(os.path.join(results_path, 'COMMIT_results.csv'))
master_kinfTable.to_csv(os.path.join(results_path, 'master_kinfTable.csv'))

log_file_path = os.path.join(results_path, 'run_log.txt')

with open(log_file_path, 'w') as file:
    for detail in run_details:
        info_text = f"""
        Day: {detail['Day']}
        Horizon: {detail['Horizon']} hours
        Load Percentage: {detail['Load Percentage']*100}%
        PHS Percentage: {detail['PHS Percentage']*100}%
        PHS Duration: {detail['PHS Duration']} hours
        Nuclear Unit: {detail['Nuclear Unit']}
        Model Name: {detail['Model Name']}
        Solar Capacity: {detail['Solar Cap']}
        Wind Capacity: {detail['Wind Cap']}
        Date of Run: {detail['Date of Run']}
        """
        file.write(info_text)
        file.write("\n" + "-"*60 + "\n")
    file.write("End of run log.\n")


print("Finished runs!")

Solved day -1, going to start solving for day 0 if keff of all reactors > 1


Executing:   0%|          | 0/10 [00:00<?, ?cell/s]

Solved day 0, going to start solving for day 1 if keff of all reactors > 1


Executing:   0%|          | 0/10 [00:00<?, ?cell/s]

Solved day 1, going to start solving for day 2 if keff of all reactors > 1


...

Solved day 358, going to start solving for day 359 if keff of all reactors > 1


Executing:   0%|          | 0/10 [00:00<?, ?cell/s]

Solved day 359, going to start solving for day 360 if keff of all reactors > 1


Executing:   0%|          | 0/10 [00:00<?, ?cell/s]

Finished runs!
